In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres, find_depth_neurons, common_utils, fit_gaussian_blob, size_control
from cottage_analysis.plotting import basic_vis_plots, plotting_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, close_loop_rsof, get_session_list
from v1_depth_analysis.v1_manuscript_2023 import common_utils as v1_common_utils

In [ ]:
VERSION = 2
SAVE_ROOT = '/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver'+str(VERSION)+'/fig3/'
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
# LOAD DATA
# Load an example session
project='hey2_3d-vision_foodres_20220101'
session_name='PZAH8.2h_S20230116'
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={'anatomical_only':3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name, flexilims_session=flexilims_session, project=None, conflicts="skip"
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

# Concatenate all sessions
results_all = v1_common_utils.concatenate_all_neurons_df(flexilims_session=flexilims_session,
                                                         session_list=get_session_list.get_all_sessions(project=project,mouse_list=[
                                                                                                                                    "PZAH6.4b",
                                                                                                                                    # "PZAG3.4f",
                                                                                                                                    "PZAH8.2h", 
                                                                                                                                    "PZAH8.2i", 
                                                                                                                                    "PZAH8.2f", 
                                                                                                                                    "PZAH10.2d", 
                                                                                                                                    "PZAH10.2f"
                                                                                                                                    ], 
                                                                                                        closedloop_only=False, 
                                                                                                        openloop_only=False),
                                                         filename="neurons_df.pickle",
                                                         cols=[
                                                             'roi','best_depth','preferred_depth_closedloop','preferred_depth_closedloop_crossval',
                                                               'depth_tuning_test_rsq_closedloop','depth_tuning_test_spearmanr_pval_closedloop',
                                                               'preferred_RS_closedloop_g2d','preferred_RS_closedloop_crossval_g2d',
                                                               'preferred_OF_closedloop_g2d','preferred_OF_closedloop_crossval_g2d',
                                                               'rsof_test_rsq_closedloop_g2d','rsof_rsq_closedloop_g2d','rsof_popt_closedloop_g2d',
                                                               'rsof_test_rsq_closedloop_gof','rsof_rsq_closedloop_gof','rsof_popt_closedloop_gof',
                                                               'rsof_test_rsq_closedloop_gadd','rsof_rsq_closedloop_gadd','rsof_popt_closedloop_gadd',
                                                               ],
                                                         
                                                         read_iscell=True,
                                                         verbose=True)
results_all["preferred_RS_closedloop_crossval_g2d"] = results_all["preferred_RS_closedloop_crossval_g2d"]*100
results_all["preferred_depth_closedloop_crossval"] = results_all["preferred_depth_closedloop_crossval"]*100
results_all["preferred_OF_closedloop_crossval_g2d"] = np.degrees(results_all["preferred_OF_closedloop_crossval_g2d"])

In [ ]:
# Fig. 3A-C: 1 example neuron, binned by RS (A), OF (B), and heatmap (C)
# Fig. 3D-F: another example neuron, binned by RS (D), OF (E), and heatmap (F)
EXAMPLE_ROIS = [
                402,
            #    86, 698,
                88,
                ]
fontsize_dict = {"title": 15, "label": 7, "tick": 5, "legend": 5}

fig = plt.figure(figsize=(7.08,7.08))
for iroi, roi in enumerate(EXAMPLE_ROIS):
    
    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=3,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0,
        plot_y=1-0.39*iroi,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict=fontsize_dict,
        )
    if iroi != len(EXAMPLE_ROIS)-1:
        plt.xlabel('')
    
    close_loop_rsof.plot_speed_tuning(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=10,
        which_speed="RS",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd = 1,
        markersize=4,
        linewidth=2,
        plot_x=0.3,
        plot_y=1-0.39*iroi,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict=fontsize_dict,
        legend_on=False,
    )
    if iroi != len(EXAMPLE_ROIS)-1:
        plt.xlabel('')
    plt.ylabel('')
    
    close_loop_rsof.plot_speed_tuning(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=10,
        which_speed="OF",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        smoothing_sd = 1,
        markersize=4,
        linewidth=2,
        plot_x=0.6,
        plot_y=1-0.39*iroi,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict=fontsize_dict,
        legend_on=True,
    )
    if iroi != len(EXAMPLE_ROIS)-1:
        plt.xlabel('')
    plt.ylabel('')
    
    if iroi != len(EXAMPLE_ROIS)-1:
        xlabel=''
    else:
        xlabel='Running speed (cm/s)'
    _ = close_loop_rsof.plot_RS_OF_matrix(fig=fig,
                                        trials_df=trials_df_example,
                                        roi=roi,
                                        log_range={
                                            "rs_bin_log_min": 0,
                                            "rs_bin_log_max": 2.5,
                                            "rs_bin_num": 6,
                                            "of_bin_log_min": -1.5,
                                            "of_bin_log_max": 3.5,
                                            "of_bin_num": 11,
                                            "log_base": 10,
                                        },
                                        xlabel=xlabel,
                                        ylabel="Optical flow speed \n(degrees/s)",
                                        plot_x=0,
                                        plot_y=0.8-0.4*iroi,
                                        plot_width=0.15,
                                        plot_height=0.15,
                                        cbar_width=0.01,
                                        fontsize_dict=fontsize_dict,
        
    )
    
    close_loop_rsof.plot_RS_OF_fitted_tuning(
        fig=fig,
        neurons_df=neurons_df_example,
        roi=roi,
        model="gaussian_2d",
        min_sigma=0.25,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        plot_x=0.25,
        plot_y=0.8-0.4*iroi,
        plot_width=0.15,
        plot_height=0.15,
        cbar_width=0.01,
        xlabel="Running speed (cm/s)",
        ylabel="Optical flow speed \n(degrees/s)",
        fontsize_dict=fontsize_dict,
    )
    
    close_loop_rsof.plot_RS_OF_fitted_tuning(
        fig=fig,
        neurons_df=neurons_df_example,
        roi=roi,
        model="gaussian_additive",
        min_sigma=0.25,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        plot_x=0.5,
        plot_y=0.8-0.4*iroi,
        plot_width=0.15,
        plot_height=0.15,
        cbar_width=0.01,
        xlabel="Running speed (cm/s)",
        ylabel="Optical flow speed \n(degrees/s)",
        fontsize_dict=fontsize_dict,
    )
    
    close_loop_rsof.plot_RS_OF_fitted_tuning(
        fig=fig,
        neurons_df=neurons_df_example,
        roi=roi,
        model="gaussian_OF",
        min_sigma=0.25,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        plot_x=0.75,
        plot_y=0.8-0.4*iroi,
        plot_width=0.15,
        plot_height=0.15,
        cbar_width=0.01,
        xlabel="Running speed (cm/s)",
        ylabel="Optical flow speed \n(degrees/s)",
        fontsize_dict=fontsize_dict,
    )
    
# Fig. : compare R2 of different models
close_loop_rsof.plot_r2_comparison(
    fig=fig,
    neurons_df=results_all,
    use_cols=["rsof_test_rsq_closedloop_g2d", "rsof_test_rsq_closedloop_gadd", "rsof_test_rsq_closedloop_gof", ],
    labels=["Conjunctive", "Additive", "OF only", ],
    plot_type="bar",
    markersize=10,
    alpha=0.3,
    color='k',
    plot_x=0,
    plot_y=0,
    plot_width=1,
    plot_height=0.3,
    fontsize_dict=fontsize_dict,
)
plt.savefig(SAVE_ROOT+'fig3a.pdf', bbox_inches='tight', transparent=True)
plt.savefig(SAVE_ROOT+'fig3a.png', bbox_inches='tight', transparent=True, dpi=300)

In [ ]:
# Fig. 3G-H: scatterplot of preferred depth vs. ratio of preferred RS and OF speeds (cross-validated)
fig=plt.figure(figsize=(7.08,7.08))
fontsize_dict = {"title": 5, "label": 7, "tick": 5, "legend": 5}
close_loop_rsof.plot_speed_depth_scatter(
    fig=fig,
    neurons_df=results_all,
    xcol="preferred_RS_closedloop_crossval_g2d",
    ycol="preferred_depth_closedloop_crossval",
    xlabel="Running speed (cm/s)",
    ylabel="Preferred depth (cm)",
    s=5,
    alpha=0.2,
    c='g',
    plot_x=0,
    plot_y=0.7,
    plot_width=0.3,
    plot_height=0.3,
    cbar_width=0.01,
    fontsize_dict=fontsize_dict,)

close_loop_rsof.plot_speed_depth_scatter(
    fig=fig,
    neurons_df=results_all,
    xcol="preferred_OF_closedloop_crossval_g2d",
    ycol="preferred_depth_closedloop_crossval",
    xlabel="Optic flow speed (degrees/s)",
    ylabel="Preferred depth (cm)",
    s=5,
    alpha=0.2,
    c='b',
    plot_x=0,
    plot_y=0.32,
    plot_width=0.5,
    plot_height=0.3,
    cbar_width=0.01,
    aspect_equal=True,
    fontsize_dict=fontsize_dict,)

results_all["RS_OF_ratio_closedloop_crossval_g2d"] = results_all["preferred_RS_closedloop_crossval_g2d"] / np.radians(results_all["preferred_OF_closedloop_crossval_g2d"])
close_loop_rsof.plot_speed_depth_scatter(
    fig=fig,
    neurons_df=results_all,
    xcol="RS_OF_ratio_closedloop_crossval_g2d",
    ycol="preferred_depth_closedloop_crossval",
    xlabel="Ratio of running speed\nto optic flow speed (cm)",
    ylabel="Preferred depth (cm)",
    s=5,
    alpha=0.2,
    c='k',
    plot_x=0,
    plot_y=0,
    plot_width=0.5,
    plot_height=0.3,
    cbar_width=0.01,
    aspect_equal=True,
    plot_diagonal=True,
    fontsize_dict=fontsize_dict,)

i=0
for of_min, of_max in zip([None, 10, 100], [10, 100, None]):
    if of_min is None:
        results_filtered = results_all[results_all["preferred_OF_closedloop_crossval_g2d"] < of_max].copy()
    elif of_max is None:
        results_filtered = results_all[results_all["preferred_OF_closedloop_crossval_g2d"] >= of_min].copy()
    else:
        results_filtered = results_all[(results_all["preferred_OF_closedloop_crossval_g2d"] >= of_min) & (results_all["preferred_OF_closedloop_crossval_g2d"] < of_max)].copy()
    if i==1:
        xlabel=''
        ylabel="Preferred depth (cm)"
    elif i==2:
        ylabel=''
        xlabel="Running speed (cm/s)"
    else:
        xlabel=''
        ylabel=''
    
    close_loop_rsof.plot_speed_depth_scatter(
        fig=fig,
        neurons_df=results_filtered,
        xcol="preferred_RS_closedloop_crossval_g2d",
        ycol="preferred_depth_closedloop_crossval",
        xlabel=xlabel,
        ylabel=ylabel,
        s=1,
        alpha=0.2,
        c='g',
        plot_x=0.4,
        plot_y=0.95-i*0.4/3,
        plot_width=0.06,
        plot_height=0.06,
        cbar_width=0.01,
        fontsize_dict=fontsize_dict,)
    i+=1
    
# Fig. 3I: scatterplot of preferred RS and OF speeds (cross-validated) colored by preferred depth
close_loop_rsof.plot_speed_colored_by_depth(
    fig=fig,
    neurons_df=results_all,
    xcol="preferred_RS_closedloop_crossval_g2d",
    ycol="preferred_OF_closedloop_crossval_g2d",
    zcol="preferred_depth_closedloop_crossval",
    xlabel="Running speed (cm/s)",
    ylabel="Optic flow speed (degree/s)",
    zlabel="Preferred depth (cm)",
    s=20,
    alpha=0.6,
    cmap='cool_r',
    plot_x=0.6,
    plot_y=0,
    plot_width=0.55,
    plot_height=1,
    cbar_width=0.01,
    fontsize_dict=fontsize_dict,
    )


plt.savefig(SAVE_ROOT+'fig3b.pdf', bbox_inches='tight', transparent=True)
plt.savefig(SAVE_ROOT+'fig3b.png', bbox_inches='tight', transparent=True, dpi=300)

In [ ]:
# CHECK:
# - what's going on with the purple cells at the bottom right corner of the scatter plot?